In [1]:
from hearing_assistive import *
from sklearn.externals import joblib
import numpy as np

%matplotlib inline

# Load Classifier

In [2]:
clf = joblib.load('SVC/svc.pk1')

# Create Testset and Test Classifier

In [3]:
alarm = read_wave('Sounds/school_fire_alarm.wav')
alarm_seg_map = alarm.partition()   # default, 2.4 second segments
alarm_seg1 = alarm_seg_map[7]
alarm_seg2 = alarm_seg_map[10]

tones = read_wave('Sounds/plectron_tones.wav')
tones_seg_map = tones.partition()   # default, 2.4 second segments
tones_seg1 = tones_seg_map[6]
tones_seg2 = tones_seg_map[5]

In [4]:
# With Noise intensity 0.01-0.02

print "Test # \t\tAccuracy (%)"
print "------------------------------"

cases = 10
total = 0
for case in range(cases):
    testset = []
    expected = []

    # add alarm sound files to the testset
    for i in range(30):
        segment = alarm_seg_map[7]
        segment = apply_filter(segment)
        spectrogram = segment.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
        dummyset = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
        testset.append(dummyset)
        expected.append(3)

    # # add tones sound files to the testset
    for i in range(30):
        segment = tones_seg_map[6]
        segment = apply_filter(segment)
        spectrogram = segment.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
        dummyset = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
        testset.append(dummyset)
        expected.append(4)

    # convert to numpy array
    testset = np.asarray(testset)

    # restructure testset before being processed by svm
    testset = testset.reshape((testset.shape[0], -1))
    
    actual = list(clf.predict(testset))
    acc = clf_accuracy(expected, actual)
    total += acc
    print "Test", case, "\t:\t", acc
    
avg = total/cases
print "------------------------------"
print "Average:\t", avg

Test # 		Accuracy (%)
------------------------------
Test 0 	:	91.6666666667
Test 1 	:	95.0
Test 2 	:	95.0
Test 3 	:	95.0
Test 4 	:	98.3333333333
Test 5 	:	93.3333333333
Test 6 	:	93.3333333333
Test 7 	:	95.0
Test 8 	:	91.6666666667
Test 9 	:	93.3333333333
------------------------------
Average:	94.1666666667


In [5]:
# With Noise intensity 0.02-0.04

print "Test # \t\tAccuracy (%)"
print "------------------------------"

cases = 10
total = 0
for case in range(cases):
    testset = []
    expected = []

    # add alarm sound files to the testset
    for i in range(30):
        segment = alarm_seg_map[7]
        segment = apply_filter(segment, factor=2)
    #     segment.plot()
        spectrogram = segment.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
        dummyset = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
        testset.append(dummyset)
        expected.append(3)

    # # add tones sound files to the testset
    for i in range(30):
        segment = tones_seg_map[6]
        segment = apply_filter(segment, factor=2)
    #     segment.plot()
        spectrogram = segment.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
        dummyset = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
        testset.append(dummyset)
        expected.append(4)

    # convert to numpy array
    testset = np.asarray(testset)

    # restructure testset before being processed by svm
    testset = testset.reshape((testset.shape[0], -1))
    
    actual = list(clf.predict(testset))
    # print "extected: \t", expected
    # print "actual: \t", actual
    acc = clf_accuracy(expected, actual)
    total += acc
    print "Test", case, "\t:\t", acc
    
avg = total/cases
print "------------------------------"
print "Average:\t", avg

Test # 		Accuracy (%)
------------------------------
Test 0 	:	76.6666666667
Test 1 	:	71.6666666667
Test 2 	:	76.6666666667
Test 3 	:	85.0
Test 4 	:	60.0
Test 5 	:	71.6666666667
Test 6 	:	78.3333333333
Test 7 	:	68.3333333333
Test 8 	:	76.6666666667
Test 9 	:	76.6666666667
------------------------------
Average:	74.1666666667


In [ ]:
# NOTES:
# When the test set does not include added noise, classifier is 100% accurate
# However, when noise is added, results may be as low as 95% if noise ranges (0.001, 0.002)
# Results may be as low as 85% if noise ranges (0.01, 0.02)
# And results may be as low as 65% if noise ranges (0.1, 0.2)